# 📈 Прогнозирование цен на недвижимость: Полный цикл ML-проекта

Этот ноутбук демонстрирует полный рабочий процесс машинного обучения на классической задаче: прогнозирование цен на жилье в Калифорнии. Мы пройдем все этапы: от загрузки и анализа данных до обучения модели, оценки ее производительности и визуализации результатов.

### Шаг 1: Импорт библиотек и загрузка данных

Подключаем все необходимые инструменты: `pandas` и `numpy` для работы с данными, `matplotlib` и `seaborn` для визуализации, и `scikit-learn` для самой модели.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score

# Загружаем датасет
housing = fetch_california_housing()

# Создаем DataFrame для удобства
data = pd.DataFrame(housing.data, columns=housing.feature_names)
data['PRICE'] = housing.target

print("Первые 5 строк данных:")
print(data.head())

print("\nОсновная информация о датасете:")
data.info()

### Шаг 2: Исследовательский анализ данных (EDA) и Визуализация

Прежде чем обучать модель, посмотрим на данные. Построим тепловую карту корреляций, чтобы увидеть, какие признаки больше всего влияют на цену.

In [ ]:
plt.style.use('dark_background')
plt.figure(figsize=(12, 8))
correlation_matrix = data.corr()
sns.heatmap(data=correlation_matrix, annot=True, cmap='coolwarm')
plt.title('Тепловая карта корреляций признаков')
plt.show()

print("\nКорреляция признаков с целевой переменной (PRICE):")
print(correlation_matrix['PRICE'].sort_values(ascending=False))

**Вывод:** Как и ожидалось, `MedInc` (средний доход) имеет самую высокую положительную корреляцию с ценой.

### Шаг 3: Подготовка данных и обучение модели

Разделяем данные на обучающую и тестовую выборки. В качестве модели будем использовать `RandomForestRegressor` — мощный и надежный ансамблевый метод.

In [ ]:
# Определяем признаки (X) и цель (y)
X = data.drop('PRICE', axis=1)
y = data['PRICE']

# Разделяем данные
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Размер обучающей выборки: {X_train.shape[0]} строк")
print(f"Размер тестовой выборки: {X_test.shape[0]} строк")

# Создаем и обучаем модель
model = RandomForestRegressor(n_estimators=100, random_state=42)
print("\nОбучение модели RandomForest...")
model.fit(X_train, y_train)
print("Обучение завершено.")

### Шаг 4: Оценка модели и анализ важности признаков

Проверяем, насколько хорошо наша модель работает на данных, которые она еще не видела. Также посмотрим, какие признаки модель посчитала самыми важными.

In [ ]:
# Делаем предсказания на тестовых данных
y_pred = model.predict(X_test)

# Оцениваем качество
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"\nСредняя абсолютная ошибка (MAE): {mae:.4f}")
print(f"Коэффициент детерминации (R²): {r2:.4f}")

# Анализ важности признаков
feature_importances = pd.Series(model.feature_importances_, index=X.columns).sort_values(ascending=False)

plt.figure(figsize=(10, 6))
sns.barplot(x=feature_importances, y=feature_importances.index, palette='viridis')
plt.title('Важность признаков')
plt.xlabel('Важность')
plt.ylabel('Признаки')
plt.show()

### Шаг 5: Визуализация результатов

Построим график, чтобы наглядно сравнить реальные цены и предсказания нашей модели. В идеале все точки должны лежать на диагональной линии.

In [ ]:
plt.figure(figsize=(10, 10))
plt.scatter(y_test, y_pred, alpha=0.3, color='cyan')
plt.plot([y.min(), y.max()], [y.min(), y.max()], '--', lw=2, color='red')
plt.xlabel('Реальная цена')
plt.ylabel('Предсказанная цена')
plt.title('Реальные vs. Предсказанные цены')
plt.show()